In [197]:
import urllib.request
import json
from pulp import *
import numpy as np

In [ ]:
# All player/team data
url1 = "https://fantasy.premierleague.com/api/bootstrap-static/"

# specific player data
# https://fantasy.premierleague.com/api/element-summary/[player_id]/

In [44]:
# download current season player data

with urllib.request.urlopen("https://fantasy.premierleague.com/api/bootstrap-static/") as url:
    data = json.loads(url.read().decode())

# Start of season picking

##  find optimal full squad for a previous season with current players and current price  

In [ ]:
# just assume that you get points from all 15 players in team throughout season
# constraints include price as well as 3 players from same team

# this finds optimal team looking at the statistic you want to maximise - eg points, points per game, creativity etc

In [143]:
# pick season to look at
season = '2018/19'

''' pick statistic to optimise. Can be one of the following
{'assists',
  'bonus',
  'bps',
  'creativity',
  'goals_scored',
  'ict_index',
  'influence',
  'minutes',
  'threat',
  'total_points'
}
'''

stat = 'total_points'

# get dictionary of player name and [corresponding id, start_of_season_cost, positon, club]
pos = {1: 'goalkeeper', 2: 'defender', 3: 'midfielder', 4: 'forward'}

player_id_dict = {x['web_name']: [x['id'],
                                  x['now_cost'] - x['cost_change_start'],
                                  pos[x['element_type']],
                                  x['team']]
                  for x in data['elements']}

In [144]:
# for each player create dict with name: [statistic, start_of_season_cost, postion, club]
# if not played then last_seasons_points is 0

last_season_dict = {}

for key in player_id_dict.keys():
    with urllib.request.urlopen("https://fantasy.premierleague.com/api/element-summary/{}/"
                                .format(player_id_dict[key][0])) as url:
        data_player = json.loads(url.read().decode())

    for x in data_player['history_past']:
        if x['season_name'] == season:
            last_season_dict[key] = [float(
                x[stat]), player_id_dict[key][1], player_id_dict[key][2], player_id_dict[key][3]]

In [172]:
player = list(last_season_dict.keys())
stat = {key: last_season_dict[key][0] for key in last_season_dict.keys()}
cost = {key: last_season_dict[key][1] for key in last_season_dict.keys()}
gk = {key: 1 if last_season_dict[key][2] ==
      'goalkeeper' else 0 for key in last_season_dict.keys()}
defe = {key: 1 if last_season_dict[key][2] ==
        'defender' else 0 for key in last_season_dict.keys()}
mid = {key: 1 if last_season_dict[key][2] ==
       'midfielder' else 0 for key in last_season_dict.keys()}
stri = {key: 1 if last_season_dict[key][2] ==
        'forward' else 0 for key in last_season_dict.keys()}

prob = LpProblem("Fantasy Football", LpMaximize)
player_vars = LpVariable.dicts("Player", player, 0, 1, LpBinary)

# objective function
prob += lpSum([stat[i]*player_vars[i] for i in player]), "Total Statistic"

# constraint
prob += lpSum([player_vars[i] for i in player]) == 15, "Total 15 Players"
prob += lpSum([cost[i] * player_vars[i]
               for i in player]) <= 1000.0, "Total Cost"
prob += lpSum([gk[i] * player_vars[i] for i in player]) == 2, "Only 1 GK"
prob += lpSum([defe[i] * player_vars[i] for i in player]) == 5, "5 DEF"
prob += lpSum([mid[i] * player_vars[i] for i in player]) == 5, "5 MID"
prob += lpSum([stri[i] * player_vars[i] for i in player]) == 3, "3 STR"

# add constraint of max 3 in a team
for i in range(20):  # 20 teams in the league
    club = {key: 1 if last_season_dict[key][3] ==
            i else 0 for key in last_season_dict.keys()}
    prob += lpSum([club[i] * player_vars[i] for i in player]
                  ) <= 3, "Less than 3 in team {}".format(i)

# solve
status = prob.solve()

In [176]:
print('total points are {} \n'.format(value(prob.objective)))
print('players in team are: ')

for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)

total points are 2545.0 

players in team are: 
Player_Alisson = 1.0
Player_Azpilicueta = 1.0
Player_Callum_Wilson = 1.0
Player_David_Luiz = 1.0
Player_Felipe_Anderson = 1.0
Player_Fraser = 1.0
Player_Jiménez = 1.0
Player_Jota = 1.0
Player_Kanté = 1.0
Player_Laporte = 1.0
Player_Milivojevic = 1.0
Player_Pickford = 1.0
Player_Robertson = 1.0
Player_Sigurdsson = 1.0
Player_van_Dijk = 1.0


## find optimal starting team for last season with current players and current price

In [179]:
# assume no transfers throughout entire season so want to maximise points in starting line up.
# constraints include price as well as 3 players from same team

# so you buy 15 players
# 2 goalkeepers, 5 defenders, 5 midfielders and 3 forwards
# you have 4 on bench (1 gk, 3 remaining players). Want to maximise points in 11 playing

In [178]:
# possible formations are 3-4-3, 3-5-2, 4-3-3, 4-4-2, 4-5-1, 5-2-3, 5-3-2 and 5-4-1
# for each formation I make the cheapest bench possible. I then optimise stat at price (1000 - price_of_bench)
# I then pick the formation with starting team that gets the most points

In [185]:
# find minimum gk, def, mid and stri values
position_stat_dict = {}

for i in range(4):
    prices_pos = [last_season_dict[key][1] for key in last_season_dict.keys()
                  if list(pos.values()).index(last_season_dict[key][2]) == i]

    position_stat_dict[i] = min(prices_pos)

In [204]:
formations = {0: [3, 4, 3], 1: [3, 5, 2], 2: [4, 3, 3], 3: [4, 4, 2],
              4: [4, 5, 1], 5: [5, 2, 3], 6: [5, 3, 2], 7: [5, 4, 1]}


team_stat_dict = {}

values = []


for key in formations.keys():
    bench_price = position_stat_dict[0] + (5 - formations[key][0])*position_stat_dict[1] + (
        5 - formations[key][1])*position_stat_dict[2] + (3 - formations[key][2])*position_stat_dict[3]

    prob = LpProblem("Fantasy Football", LpMaximize)
    player_vars = LpVariable.dicts("Player", player, 0, 1, LpBinary)

    # objective function
    prob += lpSum([stat[i]*player_vars[i] for i in player]), "Total Statistic"

    # constraint
    prob += lpSum([player_vars[i] for i in player]) == 11, "Total 11 Players"
    prob += lpSum([cost[i] * player_vars[i] for i in player]
                  ) <= (1000.0 - bench_price), "Total Cost"
    prob += lpSum([gk[i] * player_vars[i] for i in player]) == 1, "Only 1 GK"
    prob += lpSum([defe[i] * player_vars[i]
                   for i in player]) == formations[key][0], "DEF"
    prob += lpSum([mid[i] * player_vars[i]
                   for i in player]) == formations[key][1], "MID"
    prob += lpSum([stri[i] * player_vars[i]
                   for i in player]) == formations[key][2], "STR"

    # add constraint of max 3 in a team
    for i in range(20):  # 20 teams in the league
        club = {key: 1 if last_season_dict[key][3] ==
                i else 0 for key in last_season_dict.keys()}
        prob += lpSum([club[i] * player_vars[i] for i in player]
                      ) <= 3, "Less than 3 in team {}".format(i)

    # solve
    status = prob.solve()

    players = [v.name for v in prob.variables() if v.varValue > 0]
    values.append(value(prob.objective))
    team_stat_dict[key] = [value(prob.objective), players]

pos_of_max = np.argmax(values)
print('best formation is {} \n'.format(formations[pos_of_max]))
print('total points are {} \n'.format(team_stat_dict[pos_of_max][0]))
print('players in team are: {}'.format(team_stat_dict[pos_of_max][1]))

best formation is [4, 5, 1] 

total points are 2079.0 

players in team are: ['Player_David_Luiz', 'Player_Ederson', 'Player_Fraser', 'Player_Jiménez', 'Player_Laporte', 'Player_Milivojevic', 'Player_Pogba', 'Player_Robertson', 'Player_Salah', 'Player_Sigurdsson', 'Player_van_Dijk']


## optimal player recommendation from last season

In [ ]:
# you specify positions in team remaining eg [goalkeeper, defender, defender], price left and statistic you want to maximise and
# this will recommend the players that last season would maximise this

# During season transfers

## in-form best 11 with unlimited price 

In [ ]:
# you specify num of previous gameweeks to look at and this shows best staring team over those gameweeks

## in-form best 11 with constrained price

In [ ]:
# you specify num of previous gameweeks to look at as well as max price of 11 players

# Data Visualisation suite